In [1]:
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from scipy.stats import chi2
import scipy.stats as stats
ds = [206,99,103,105,104,489,108]

In [2]:
df = pd.read_excel('df.xlsx')

# Filter to just KHV rows
khv = df[df['Ethnic'] == 'KHV206g'] 

# Filter to just CYP2B6 rows
cyp3a5 = khv[khv['Gene'] == 'CYP3A5']

# Get unique CYP2B6 alleles 
cyp3a5_alleles = set(cyp3a5['Star Allele 1'].unique()) | set(cyp3a5['Star Allele 2'].unique())

print(cyp3a5_alleles)

{1, 3, 4}


In [3]:
cyp3a5_alleles = list(cyp3a5_alleles)
cyp3a5_alleles = [x for x in cyp3a5_alleles if str(x) != 'nan']

print(cyp3a5_alleles)


cyp3a5table = pd.DataFrame(cyp3a5_alleles, columns=['CYP3A5'])
cyp3a5table['KHV'] = 0.0
print(cyp3a5table)

[1, 3, 4]
   CYP3A5  KHV
0       1  0.0
1       3  0.0
2       4  0.0


In [4]:
khv_cyp3a5 = df.query("Ethnic == 'KHV206g' & Gene == 'CYP3A5'")

print(khv_cyp3a5)
start = 0
for allele in cyp3a5_alleles:
  res = 0
  for index, row in khv_cyp3a5.iterrows():
    if allele == row['Star Allele 1']:
      res += row['Percentage per allele']
    if allele == row['Star Allele 2']:  
      res += row['Percentage per allele']
  cyp3a5table.at[start,'KHV'] = res
  start += 1


to_drop = cyp3a5table[cyp3a5table['KHV'] < 1]

# Drop those rows by index
cyp3a5table.drop(index=to_drop.index, inplace=True)
# Divide by 100  
print(cyp3a5table)

      Ethnic            Alleles  Percentage    Gene  Star Allele 1  \
443  KHV206g  CYP3A5*1/CYP3A5*1     11.6500  CYP3A5              1   
444  KHV206g  CYP3A5*1/CYP3A5*3     37.8600  CYP3A5              1   
445  KHV206g  CYP3A5*1/CYP3A5*4      0.9709  CYP3A5              1   
446  KHV206g  CYP3A5*3/CYP3A5*3     45.6300  CYP3A5              3   

     Star Allele 2  Percentage per allele  
443              1                5.82500  
444              3               18.93000  
445              4                0.48545  
446              3               22.81500  
   CYP3A5       KHV
0       1  31.06545
1       3  64.56000


In [5]:
temp = cyp3a5table['CYP3A5'].values.tolist()

ethnics = df['Ethnic'].unique()
ethnics =np.delete(ethnics, -1)
value_to_delete = 'KHV'
idx = np.where(ethnics == value_to_delete)[0][0]
ethnics = np.delete(ethnics, idx)
print(ethnics)


cyp3a5_rows = df[df['Gene'] == 'CYP3A5']

# Create bool series to check if allele in temp
allele1_matches = cyp3a5_rows['Star Allele 1'].isin(temp)
allele2_matches = cyp3a5_rows['Star Allele 2'].isin(temp)

# Filter for rows where either allele matches 
mask = allele1_matches | allele2_matches
cyp3a5_population = cyp3a5_rows[mask]
cyp3a5_population = cyp3a5_population[cyp3a5_population['Ethnic'] != 'KHV206g']

print(cyp3a5_population)#print(temp)

['CEU' 'CHB' 'CHS' 'JPT' 'SAS' 'YRI']
    Ethnic            Alleles  Percentage    Gene  Star Allele 1  \
330    CEU  CYP3A5*1/CYP3A5*3      8.0810  CYP3A5              1   
331    CEU  CYP3A5*3/CYP3A5*3     89.9000  CYP3A5              3   
332    CHB  CYP3A5*1/CYP3A5*1     11.6500  CYP3A5              1   
333    CHB  CYP3A5*1/CYP3A5*3     38.8300  CYP3A5              1   
334    CHB  CYP3A5*3/CYP3A5*3     49.5100  CYP3A5              3   
335    CHS  CYP3A5*1/CYP3A5*1      7.6190  CYP3A5              1   
336    CHS  CYP3A5*1/CYP3A5*3     38.1000  CYP3A5              1   
337    CHS  CYP3A5*3/CYP3A5*3     49.5200  CYP3A5              3   
338    KHV  CYP3A5*1/CYP3A5*1      7.0710  CYP3A5              1   
339    KHV  CYP3A5*1/CYP3A5*3     41.4100  CYP3A5              1   
340    KHV  CYP3A5*3/CYP3A5*3     48.4800  CYP3A5              3   
341    JPT  CYP3A5*1/CYP3A5*1      3.8460  CYP3A5              1   
342    JPT  CYP3A5*1/CYP3A5*3     43.2700  CYP3A5              1   
343    JPT

In [6]:
for ethnic in ethnics:
        cyp3a5table[ethnic] = 0.0

cyp3a5table = cyp3a5table.reset_index(drop=True)

print(cyp3a5table)

   CYP3A5       KHV  CEU  CHB  CHS  JPT  SAS  YRI
0       1  31.06545  0.0  0.0  0.0  0.0  0.0  0.0
1       3  64.56000  0.0  0.0  0.0  0.0  0.0  0.0


In [7]:
start = 0
for ethnic in ethnics:
    start = 0
    for starallele in temp:
        res = 0

        for index, row in cyp3a5_population.iterrows():
            if row['Ethnic'] == ethnic:
                if starallele == row['Star Allele 1']:
                    res += row['Percentage per allele']
                if starallele == row['Star Allele 2']:  
                    res += row['Percentage per allele']
                cyp3a5table.at[start,ethnic] = res
        #print(start)    
        start += 1
    cyp3a5table[ethnic] = cyp3a5table[ethnic].round(1)

cyp3a5table['KHV'] = cyp3a5table['KHV'].round(1)
cyp3a5table.to_csv('frq/cyp3a5.csv')
print(cyp3a5table)

   CYP3A5   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0       1  31.1   4.0  31.1  26.7  25.5  33.2  51.4
1       3  64.6  93.9  68.9  68.6  74.5  66.8  14.4


In [8]:
init = {'CEU': [0], 'CHB': [0], 'CHS': [0], 'JPT': [0], 'SAS': [0], 'YRI': [0]}
gst = pd.DataFrame(init, dtype=float)

h_s_k = 1.0
for index,value in cyp3a5table.iterrows():
    h_s_k -= (cyp3a5table.at[index,'KHV']/100) * (cyp3a5table.at[index,'KHV']/100)

for idx in range(0,6):
    value = 0
    h_t = 1.0
    h_s = 1.0
    for id in range(0,2):
        h_t -= ((cyp3a5table.iloc[id,idx + 2] + cyp3a5table.iloc[id,1])/200) * ((cyp3a5table.iloc[id,idx + 2] + cyp3a5table.iloc[id,1])/200)
        h_s -= (cyp3a5table.iloc[id,idx + 2]/100) * (cyp3a5table.iloc[id,idx + 2]/100)
    h_s = (h_s + h_s_k)/2
    gst.iloc[0,idx] = (h_t - h_s)/h_t # You can modify the value here'''
gst = gst.round(4)
gst.index = ['CYP3A5']
gst.to_csv('gst/gst_cyp3a5.csv')
print(gst)

           CEU    CHB     CHS     JPT     SAS     YRI
CYP3A5  0.1167  0.001  0.0019  0.0074  0.0005  0.1088


In [9]:
test = list(cyp3a5table['CYP3A5'])
test = ['CYP3A5*' + str(int(item)) for item in test]

def generate_gene_combinations(alleles):
    res = []
    length = len(alleles)
    for x in range(0,length):
        for y in range(x,length):
            lol = f'{alleles[x]}/{alleles[y]}'
            res.append(lol)

    return res

print(generate_gene_combinations(test))
gene_combinations = generate_gene_combinations(test)

degrees_of_freedom = len(gene_combinations) - 1
print(gene_combinations)

['CYP3A5*1/CYP3A5*1', 'CYP3A5*1/CYP3A5*3', 'CYP3A5*3/CYP3A5*3']
['CYP3A5*1/CYP3A5*1', 'CYP3A5*1/CYP3A5*3', 'CYP3A5*3/CYP3A5*3']


In [10]:
observed = pd.DataFrame(gene_combinations, columns=['Gene'])
new_columns = ['KHV', 'CEU', 'CHB', 'CHS', 'JPT', 'SAS', 'YRI']
for column in new_columns:
    observed[column] = 0.0

print(observed)

                Gene  KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  CYP3A5*1/CYP3A5*1  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  CYP3A5*1/CYP3A5*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  CYP3A5*3/CYP3A5*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [11]:
l = len(khv_cyp3a5)
for x in range(0,l):
    if khv_cyp3a5.iloc[x,1] in gene_combinations:
        gene_to_update = khv_cyp3a5.iloc[x,1]
        new_value = khv_cyp3a5.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, 'KHV'] = new_value

length = len(cyp3a5_population)
for x in range(0,length):
    if cyp3a5_population.iloc[x,1] in gene_combinations:
        gene_to_update = cyp3a5_population.iloc[x,1]
        pop_to_update = cyp3a5_population.iloc[x,0]
        new_value = cyp3a5_population.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, pop_to_update ] = new_value 

for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        observed.iloc[y,x+1] = observed.iloc[y,x+1] * ((ds[x] * 1.0)/ 100)

observed = observed.round(0)
print(observed)

                Gene    KHV   CEU   CHB   CHS   JPT    SAS   YRI
0  CYP3A5*1/CYP3A5*1   15.0   0.0  12.0   8.0   4.0   59.0  34.0
1  CYP3A5*1/CYP3A5*3   85.0   8.0  40.0  40.0  45.0  207.0  22.0
2  CYP3A5*3/CYP3A5*3  100.0  89.0  51.0  52.0  55.0  223.0   1.0


In [12]:
hypo = pd.DataFrame(gene_combinations, columns=['Gene'])
for column in new_columns:
    hypo[column] = 0.0

allele = cyp3a5table.iloc[:,0].tolist()

for x in range(0,7):
    temp = cyp3a5table.iloc[:,x + 1].tolist()
    for allele1 in range(0,len(temp)):
        for allele2 in range(allele1,len(temp)):
            if allele1 == allele2:
                new_value = (temp[allele1] / 100) * (temp[allele2] / 100)
            elif allele1 != allele2:
                new_value = 2 * (temp[allele1] / 100) * (temp[allele2] / 100)
            gene = f"CYP3A5*{int(allele[allele1])}/CYP3A5*{int(allele[allele2])}"
            
            row_index = hypo.index[hypo['Gene'] == gene].tolist()[0]
            hypo.iloc[row_index, x+1] = new_value

print(hypo)

                Gene       KHV       CEU       CHB       CHS       JPT  \
0  CYP3A5*1/CYP3A5*1  0.096721  0.001600  0.096721  0.071289  0.065025   
1  CYP3A5*1/CYP3A5*3  0.401812  0.075120  0.428558  0.366324  0.379950   
2  CYP3A5*3/CYP3A5*3  0.417316  0.881721  0.474721  0.470596  0.555025   

        SAS       YRI  
0  0.110224  0.264196  
1  0.443552  0.148032  
2  0.446224  0.020736  


In [13]:
for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        hypo.iloc[y,x+1] = hypo.iloc[y,x+1] * ((ds[x] * 1.0))
        if hypo.iloc[y,x+1] < 1:
            hypo.iloc[y,x+1] = 1
hypo = hypo.round(0)
print(hypo)

                Gene   KHV   CEU   CHB   CHS   JPT    SAS   YRI
0  CYP3A5*1/CYP3A5*1  20.0   1.0  10.0   7.0   7.0   54.0  29.0
1  CYP3A5*1/CYP3A5*3  83.0   7.0  44.0  38.0  40.0  217.0  16.0
2  CYP3A5*3/CYP3A5*3  86.0  87.0  49.0  49.0  58.0  218.0   2.0


In [14]:
hihi = {"KHV": [0.0], "CEU": [0.0], "CHB": [0.0], "CHS": [0.0], "JPT": [0.0], "SAS": [0.0], "YRI": [0.0]}
chi_square = pd.DataFrame(hihi)
print(chi_square)
for x in range(0,7):
    res = 0
    for y in range(0,len(hypo)):
        chi_square_value = ((hypo.iloc[y,x+1] - observed.iloc[y,x+1]) * (hypo.iloc[y,x+1] - observed.iloc[y,x+1])) / (hypo.iloc[y,x+1])
        res += chi_square_value

    p_value = 1 - stats.chi2.cdf(res, degrees_of_freedom)
    chi_square.iloc[0,x] = p_value 

chi_square.index = ['CYP3A5']
chi_square = chi_square.round(4)
print(chi_square)

   KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
           KHV     CEU     CHB     CHS    JPT    SAS     YRI
CYP3A5  0.1672  0.5519  0.6553  0.8058  0.356  0.595  0.1643


In [ ]:
print(cyp3a5table)
def f(row):
    return "CYP3A5*" + str(row)
cyp3a5table["CYP3A5"] = cyp3a5table["CYP3A5"].apply(f)
cyp3a5table.rename(columns={"CYP3A5": "Gene"}, inplace=True)
cyp3a5table.to_excel("heatmap/cyp3a5table.xlsx")